# 🎬 SmartClip AI - Google Colab

Transform long-form videos into viral-ready short clips using Google Gemini AI.

**Features:**
- 🤖 AI-powered viral moment detection
- 📝 Automatic word-level captions
- 🎯 Smart face tracking & active speaker detection
- 📺 Support for YouTube URLs & local files

---

## 📋 Instructions

1. **Run Cell 1** - Setup & Install dependencies
2. **Run Cell 2** - Enter your Gemini API key
3. **Run Cell 3** - Upload video or enter YouTube URL
4. **Run Cell 4** - Configure options & generate clips
5. **Run Cell 5** - Download your clips

---

## 1️⃣ Setup & Installation

Run this cell first to install all dependencies.

In [ ]:
#@title 🔧 Install SmartClip AI
#@markdown This will clone the repository and install all dependencies.

import os
import subprocess

# Clone repository
if not os.path.exists('/content/sclip'):
    print("📥 Cloning SmartClip AI repository...")
    !git clone https://github.com/sakirsyarian/sclip.git /content/sclip
else:
    print("📁 Repository already exists, pulling latest changes...")
    !cd /content/sclip && git pull

# Change to project directory
os.chdir('/content/sclip')

# Install dependencies
print("\n📦 Installing dependencies...")
!pip install -q -r requirements.txt
!pip install -q -e .

# Update yt-dlp to latest version (fixes YouTube 403 errors)
print("\n🔄 Updating yt-dlp to latest version...")
!pip install -q -U yt-dlp

# Verify FFmpeg (pre-installed on Colab)
print("\n🔍 Checking FFmpeg...")
!ffmpeg -version | head -1

# Check yt-dlp version
print("\n🔍 Checking yt-dlp...")
!yt-dlp --version

# Check GPU availability
print("\n🖥️ Checking GPU...")
try:
    import torch
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        print(f"✅ GPU available: {gpu_name}")
    else:
        print("⚠️ No GPU detected. Face tracking will use CPU (slower but works).")
except:
    print("ℹ️ PyTorch not installed. Face tracking will use CPU.")

print("\n✅ Setup complete! Proceed to the next cell.")

## 2️⃣ Configure API Key

Get your free API key from [Google AI Studio](https://aistudio.google.com/apikey)

In [ ]:
#@title 🔑 Enter Gemini API Key
#@markdown Get your free API key from [Google AI Studio](https://aistudio.google.com/apikey)

from getpass import getpass
import os

# Option 1: Use Colab Secrets (recommended)
try:
    from google.colab import userdata
    api_key = userdata.get('GEMINI_API_KEY')
    if api_key:
        print("✅ API key loaded from Colab Secrets")
except:
    api_key = None

# Option 2: Manual input
if not api_key:
    api_key = getpass("Enter your Gemini API key: ")

if api_key:
    os.environ['GEMINI_API_KEY'] = api_key
    masked = api_key[:4] + "..." + api_key[-4:]
    print(f"✅ API key configured: {masked}")
else:
    print("❌ No API key provided. Please enter your API key.")

## 3️⃣ Upload Video

Choose one of the options below:
- **Option A**: Upload from your computer
- **Option B**: Use YouTube URL
- **Option C**: Use Google Drive

In [ ]:
#@title 📤 Upload Video
#@markdown Choose your video source:

video_source = "Upload from computer" #@param ["Upload from computer", "YouTube URL", "Google Drive"]
youtube_url = "" #@param {type:"string"}
drive_path = "/content/drive/MyDrive/video.mp4" #@param {type:"string"}

import os

video_path = None

if video_source == "Upload from computer":
    from google.colab import files
    print("📤 Please select a video file to upload...")
    uploaded = files.upload()
    if uploaded:
        filename = list(uploaded.keys())[0]
        video_path = f"/content/sclip/{filename}"
        # Move to project directory
        os.rename(f"/content/{filename}" if os.path.exists(f"/content/{filename}") else filename, video_path)
        print(f"✅ Uploaded: {filename}")
    else:
        print("❌ No file uploaded")

elif video_source == "YouTube URL":
    if youtube_url:
        video_path = youtube_url
        print(f"✅ YouTube URL: {youtube_url}")
    else:
        print("❌ Please enter a YouTube URL in the field above")

elif video_source == "Google Drive":
    from google.colab import drive
    print("📁 Mounting Google Drive...")
    drive.mount('/content/drive')
    if os.path.exists(drive_path):
        video_path = drive_path
        print(f"✅ Found: {drive_path}")
    else:
        print(f"❌ File not found: {drive_path}")
        print("Please update the drive_path variable above.")

# Store for next cell
if video_path:
    os.environ['SCLIP_VIDEO_PATH'] = video_path
    print(f"\n🎬 Video ready: {video_path}")

## 4️⃣ Generate Clips

Configure options and run SmartClip AI.

In [ ]:
#@title 🎬 Generate Clips
#@markdown Configure your clip settings:

max_clips = 5 #@param {type:"slider", min:1, max:10, step:1}
aspect_ratio = "9:16" #@param ["9:16", "1:1", "16:9"]
caption_style = "default" #@param ["default", "bold", "minimal", "karaoke"]
language = "id" #@param ["id", "en", "es", "fr", "de", "ja", "ko", "zh"]
min_duration = 45 #@param {type:"slider", min:15, max:60, step:5}
max_duration = 180 #@param {type:"slider", min:60, max:300, step:15}
no_captions = False #@param {type:"boolean"}
dry_run = False #@param {type:"boolean"}

import os
import subprocess

# Get video path from previous cell
video_path = os.environ.get('SCLIP_VIDEO_PATH', '')

if not video_path:
    print("❌ No video selected. Please run Cell 3 first.")
else:
    # Build command
    os.chdir('/content/sclip')
    
    # Determine if URL or file
    if video_path.startswith('http'):
        input_arg = f'-u "{video_path}"'
    else:
        input_arg = f'-i "{video_path}"'
    
    cmd = f"""sclip {input_arg} \
        -n {max_clips} \
        -a {aspect_ratio} \
        -s {caption_style} \
        -l {language} \
        --min-duration {min_duration} \
        --max-duration {max_duration} \
        -o /content/sclip/output \
        -f -v"""
    
    if no_captions:
        cmd += " --no-captions"
    if dry_run:
        cmd += " --dry-run"
    
    print("🚀 Running SmartClip AI...")
    print(f"\n📋 Command: {cmd}\n")
    print("=" * 60)
    
    # Run command
    !{cmd}
    
    print("\n" + "=" * 60)
    
    # List output files
    output_dir = '/content/sclip/output'
    if os.path.exists(output_dir):
        files = [f for f in os.listdir(output_dir) if f.endswith('.mp4')]
        if files:
            print(f"\n✅ Generated {len(files)} clip(s):")
            for f in files:
                size = os.path.getsize(os.path.join(output_dir, f)) / (1024*1024)
                print(f"   📹 {f} ({size:.1f} MB)")
        else:
            print("\n⚠️ No clips generated. Check the output above for errors.")
    else:
        print("\n⚠️ Output directory not found.")

## 5️⃣ Download Clips

Download your generated clips.

In [ ]:
#@title 📥 Download Clips
#@markdown Choose download method:

download_method = "Download to computer" #@param ["Download to computer", "Save to Google Drive", "Preview in notebook"]
drive_output_folder = "/content/drive/MyDrive/SmartClip_Output" #@param {type:"string"}

import os
import shutil

output_dir = '/content/sclip/output'

if not os.path.exists(output_dir):
    print("❌ No output directory found. Please run Cell 4 first.")
else:
    files = [f for f in os.listdir(output_dir) if f.endswith('.mp4')]
    
    if not files:
        print("❌ No clips found in output directory.")
    else:
        if download_method == "Download to computer":
            from google.colab import files as colab_files
            print(f"📥 Downloading {len(files)} clip(s)...")
            for f in files:
                filepath = os.path.join(output_dir, f)
                print(f"   Downloading: {f}")
                colab_files.download(filepath)
            print("\n✅ Download complete!")
            
        elif download_method == "Save to Google Drive":
            from google.colab import drive
            drive.mount('/content/drive', force_remount=False)
            
            # Create output folder
            os.makedirs(drive_output_folder, exist_ok=True)
            
            print(f"📁 Copying {len(files)} clip(s) to Google Drive...")
            for f in files:
                src = os.path.join(output_dir, f)
                dst = os.path.join(drive_output_folder, f)
                shutil.copy2(src, dst)
                print(f"   ✅ {f}")
            
            # Also copy metadata files
            txt_files = [f for f in os.listdir(output_dir) if f.endswith('.txt')]
            for f in txt_files:
                src = os.path.join(output_dir, f)
                dst = os.path.join(drive_output_folder, f)
                shutil.copy2(src, dst)
            
            print(f"\n✅ Saved to: {drive_output_folder}")
            
        elif download_method == "Preview in notebook":
            from IPython.display import HTML, display
            import base64
            
            print(f"🎬 Previewing {len(files)} clip(s)...\n")
            
            for f in files:
                filepath = os.path.join(output_dir, f)
                
                # Read and encode video
                with open(filepath, 'rb') as video_file:
                    video_data = base64.b64encode(video_file.read()).decode('utf-8')
                
                # Display video
                print(f"📹 {f}")
                display(HTML(f'''
                <video width="300" controls>
                    <source src="data:video/mp4;base64,{video_data}" type="video/mp4">
                </video>
                '''))
                print()

## 🧹 Cleanup (Optional)

Remove temporary files to free up space.

In [ ]:
#@title 🧹 Cleanup
#@markdown Remove temporary files and free up space.

delete_output = False #@param {type:"boolean"}
delete_uploaded_video = False #@param {type:"boolean"}
delete_repository = False #@param {type:"boolean"}

import os
import shutil

if delete_output:
    output_dir = '/content/sclip/output'
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
        print("✅ Deleted output directory")
    else:
        print("ℹ️ Output directory not found")

if delete_uploaded_video:
    video_path = os.environ.get('SCLIP_VIDEO_PATH', '')
    if video_path and os.path.exists(video_path) and not video_path.startswith('http'):
        os.remove(video_path)
        print(f"✅ Deleted: {video_path}")
    else:
        print("ℹ️ No uploaded video to delete")

if delete_repository:
    os.chdir('/content')
    if os.path.exists('/content/sclip'):
        shutil.rmtree('/content/sclip')
        print("✅ Deleted repository")
    else:
        print("ℹ️ Repository not found")

if not any([delete_output, delete_uploaded_video, delete_repository]):
    print("ℹ️ No cleanup options selected. Check the boxes above to delete files.")

---

## 📚 Tips & Troubleshooting

### YouTube Download Error (403 Forbidden)
- This happens when YouTube blocks Colab's IP address
- **Solution 1**: Re-run Cell 1 to update yt-dlp to latest version
- **Solution 2**: Upload video manually instead of using YouTube URL
- **Solution 3**: Download video to Google Drive first, then use Drive option

### Rate Limits (Error 429)
- Gemini free tier has rate limits. Wait a few minutes and try again.
- Use `dry_run = True` to preview clips without rendering.

### Video Requirements
- Minimum duration: 60 seconds
- Supported formats: MP4, MKV, AVI, MOV, WebM
- Videos > 30 minutes are automatically chunked

### Performance
- Enable GPU runtime for faster face tracking: Runtime → Change runtime type → T4 GPU
- Colab provides ~12GB RAM and 15GB disk space

### Links
- [GitHub Repository](https://github.com/sakirsyarian/sclip)
- [Get Gemini API Key](https://aistudio.google.com/apikey)
- [Report Issues](https://github.com/sakirsyarian/sclip/issues)

---

Made with ❤️ by SmartClip AI